In [2]:
import pandas as pd
import itertools

In [26]:
df = pd.read_excel('risk_dataset.xlsx')
df.head()

,accountid,Transaction Status,fraud,TransactionDate,OrderID,PaymentInstrumentAddedDate,Payment Instrument Region,Microsoft Account Created Date,Microsoft Account Email,ProductName,s_Locale,User Agent,IPAddress,IP Region,IP Country,Shopping Cart Price USD
0,482947,"RISK APPROVED, BANK APPROVED",nonfraud,2018-07-08 18:57:00,9153744037,2018-05-13 16:14:00,ohio,2018-05-13 16:08:00,14.******j@gmail.com,Xbox Live 12-Month Gold Membership,en-US,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_4)...,66.61.xxx.xxx,ohio,United States,59.99
1,482950,"RISK APPROVED, BANK APPROVED",nonfraud,2018-07-08 02:11:00,16535293341,2018-07-08 02:10:00,maine,2018-07-08 02:08:00,603******35.cw@gmail.com,Xbox Live 3-Month Gold Membership,en-US,Mozilla/5.0 (Linux; U; Android 4.4.2; en-us; L...,66.30.xxx.xxx,maine,United States,24.99
2,482953,"RISK APPROVED, BANK APPROVED",nonfraud,2018-07-08 07:35:00,9151889337,2018-07-08 07:34:00,colorado,2018-07-08 07:33:00,aar******hiudh833@outlook.com,Xbox Live 3-Month Gold Membership|Xbox Live 3-...,en-US,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:18.0) G...,134.124.xxx.xxx,missouri,United States,74.97
3,482956,RISK REJECTED,fraud,2018-07-08 03:05:00,16535465841,2018-07-08 03:04:00,rhode island,2018-07-08 02:57:00,abb******1@outlook.com,$50 Skype Credit,en-US,Mozilla/5.0 (Windows NT 5.1; rv:27.0) Gecko/20...,96.253.xxx.xxx,rhode island,United States,50.00
4,482959,RISK REJECTED,fraud,2018-07-08 07:10:00,9151838537,2018-07-08 07:09:00,alberta,2018-07-08 07:08:00,ada******@outlook.com,Xbox Live 3-Month Gold Membership|Xbox Live 3-...,en-US,Mozilla/5.0 (Windows NT 5.1; rv:24.0) Gecko/20...,134.124.xxx.xxx,missouri,United States,74.97


In [39]:
cols = ['Payment Instrument Region', 'IP Region', 'IP Country']
cols

['Payment Instrument Region', 'IP Region', 'IP Country']

In [27]:
def fraud_rate(df, agg, cols=None, threshold=None, limit=None, days=None):
    if isinstance(cols, str):
        groupcols = [cols]
    elif cols is None:
        groupcols = [] 
    else:
        try:
            groupcols = list(cols)
        except:
            raise TypeError('Unable to convert cols to a list.')
    fraudcols = groupcols + ['fraud']
    cnts = df.groupby(fraudcols)[agg].nunique()
    if groupcols:
        wide = cnts.unstack('fraud').fillna(0)
    else:
        wide = cnts.to_frame().stack(level=0).unstack('fraud').fillna(0)
    wide = wide.assign(fraud_rate = wide.fraud / (wide.fraud + wide.nonfraud))
    if threshold:
        wide = wide.loc[wide.nonfraud.gt(threshold)]
    wide = wide.sort_values('fraud_rate', ascending=False)
    if limit:
        wide = wide.head(n=limit)
    wide = wide.assign(fpr=wide.nonfraud / wide.fraud)
    if days:
        wide = wide.assign(impact = (wide.nonfraud + wide.fraud) / days)
    return wide

In [47]:
fraud_rate = fraud_rate(df,'accountid',cols)
print(fraud_rate.shape)
fraud_rate.head(20)

(248, 4)


,,fraud,fraud,nonfraud,fraud_rate,fpr
Payment Instrument Region,IP Region,IP Country,,,,
wyoming,ha noi,Vietnam,1.0,0.0,1.0,0.0
new jersey,michigan,United States,3.0,0.0,1.0,0.0
kentucky,california,United States,1.0,0.0,1.0,0.0
kansas,ha noi,Vietnam,1.0,0.0,1.0,0.0
new jersey,minnesota,United States,1.0,0.0,1.0,0.0
texas,georgia,United States,1.0,0.0,1.0,0.0
gwynedd,gwynedd,United Kingdom,3.0,0.0,1.0,0.0
texas,ha noi,Vietnam,2.0,0.0,1.0,0.0
new york,minnesota,United States,1.0,0.0,1.0,0.0


In [42]:
def multiple_defect(nds, agg, cols, n, minrate=0.05):
    import itertools
    combos = list(itertools.combinations(cols, n))
    num_cols = ['col' + str(i) for i in range(1, n + 1)]
    res = []
    for combo in combos:
        fr = fraud_rate(nds, agg, combo)
        fr = fr.reset_index()
        fr.columns = num_cols + ['fraud', 'nonfraud', 'fraudrate','test']
        fr = fr.assign(groupcols = ', '.join(combo))
        fr = fr.loc[fr.fraudrate.gt(minrate)]
        res.append(fr)
    return pd.concat(res).sort_values(by='fraudrate', ascending=False)

In [45]:
multiple_defect(df,'accountid',cols,3,.9).head(20)

,col1,col2,col3,fraud,nonfraud,fraudrate,test,groupcols
0,wyoming,ha noi,Vietnam,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
41,michigan,arizona,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
59,west virginia,oregon,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
58,washington,minnesota,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
57,washington,michigan,United States,2.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
56,west virginia,florida,United States,2.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
55,new york,maryland,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
54,west virginia,georgia,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
53,west virginia,ha noi,Vietnam,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"
52,alberta,california,United States,1.0,0.0,1.0,0.0,"Payment Instrument Region, IP Region, IP Country"


# test code

In [34]:
def multiple_defect3(nds, agg, cols, n, count_fraud, minrate=0):
    combos = list(itertools.combinations(cols,n))
    num_cols = ['col1','col2','col3']
    res = []
    for combo in combos:
        fr = fraud_rate(nds, agg, combo)
        accounts = nds.groupby(combo).apply(lambda df: list(df.accountid.unique()))
        accounts.name = 'accounts'
        accounts = accounts.to_frame()
        fr = accounts.join(fr) #defaults to merge type = 'inner' on index
        fr = fr.reset_index()
        print(fr.columns)
        fr['naccts'] = fr.apply(lambda df: len(set(df.accounts)), axis=1)
        fr.columns = num_cols + ['accounts','naccts','fraud','nonfraud','fraudrate','fpr']
        fr = fr.assign(groupcols = ', '.join(combo))
        fr = fr[fr['fraud'] >= count_fraud]
        fr = fr.loc[fr.fraudrate.gt(minrate)]
        res.append(fr)
        time.sleep(0.2)
    return pd.concat(res).sort_values(by='fraudrate', ascending=False)

In [35]:
fraud_combos_3 = multiple_defect3(df,'accountid',cols,3,1,.5)
fraud_combos_3

KeyError: ('Payment Instrument Region', 'IP Region', 'IP Country')

In [ ]:
import os

import zelleio.read as zread

TARGET_PATH = '/sas_work5/InternalProject/frm/models/targets/'

def fraud_rate(df, agg, cols=None, threshold=None, limit=None, days=None):
    if isinstance(cols, str):
        groupcols = [cols]
    elif cols is None:
        groupcols = [] 
    else:
        try:
            groupcols = list(cols)
        except:
            raise TypeError('Unable to convert cols to a list.')
    fraudcols = groupcols + ['fraud']
    cnts = df.groupby(fraudcols)[agg].nunique()
    if groupcols:
        wide = cnts.unstack('fraud').fillna(0)
    else:
        wide = cnts.to_frame().stack(level=0).unstack('fraud').fillna(0)
    wide = wide.assign(fraud_rate = wide.fraud / (wide.fraud + wide.nonfraud))
    if threshold:
        wide = wide.loc[wide.nonfraud.gt(threshold)]
    wide = wide.sort_values('fraud_rate', ascending=False)
    if limit:
        wide = wide.head(n=limit)
    wide = wide.assign(fpr=wide.nonfraud / wide.fraud)
    if days:
        wide = wide.assign(impact = (wide.nonfraud + wide.fraud) / days)
    return wide


def error_rate(df, agg, cols=None, threshold=None, limit=None, days=None):
    if isinstance(cols, str):
        groupcols = [cols]
    elif cols is None:
        groupcols = []
    else:
        try:
            groupcols = list(cols)
        except:
            raise TypeError('Unable to convert cols to a list.')
    whitelistedcols = groupcols + ['whitelisted']
    cnts = df.groupby(whitelistedcols)[agg].nunique()
    if groupcols:
        wide = cnts.unstack('whitelisted').fillna(0)
    else:
        wide = cnts.to_frame().stack(level=0).unstack('whitelisted').fillna(0)
    wide = wide.assign(error_rate = wide.whitelisted / (wide.whitelisted + wide.nonwhitelisted))
    if threshold:
        wide = wide.loc[wide.nonwhitelisted.gt(threshold)]
    wide = wide.sort_values('error_rate', ascending=False)
    if limit:
        wide = wide.head(n=limit)
    wide = wide.assign(fpr=wide.nonwhitelisted / wide.whitelisted)
    if days:
        wide = wide.assign(impact = (wide.nonwhitelisted + wide.whitelisted) / days)
    return wide


def join_fraud(df, key, source=None):
    import zelleio
    fl = zelleio.read.read_fraud_lookup()
    if source:
        if isinstance(source, list):
            fraud = fl.loc[fl.source.isin(source), [key]].assign(fraud = 'fraud')
        else:
            fraud = fl.loc[fl.source.eq(source), [key]].assign(fraud = 'fraud')
    else:
        fraud = fl.loc[:, [key]].assign(fraud = 'fraud')
    res = df.merge(fraud, on=key, how='left')
    res.loc[res.fraud.isnull(), 'fraud'] = 'nonfraud'
    return res
    

def multiple_defect(nds, agg, cols, n, minrate=0.05):
    import itertools
    combos = list(itertools.combinations(cols, 2))
    num_cols = ['col' + str(i) for i in range(1, n + 1)]
    res = []
    for combo in combos:
        fr = fraud_rate(nds, agg, combo)
        fr = fr.reset_index()
        fr.columns = num_cols + ['fraud', 'nonfraud', 'fraudrate']
        fr = fr.assign(groupcols = ', '.join(combo))
        fr = fr.loc[fr.fraudrate.gt(minrate)]
        res.append(fr)
    return pd.concat(res).sort_values(by='fraudrate', ascending=False)


def fraud_mask(rid, accountid, key, source = None):
    # import zelleio
    # import numpy
    # fl = zelleio.read.read_fraud_lookup()
    # return 0    
    pass


def fraud_zfc_rids():
    return zread.read_hdfs(os.path.join(TARGET_PATH, 'target_zfc'))

def fraud_disp_rids():
    return zread.read_hdfs(os.path.join(TARGET_PATH, 'target_disp'))

def fraud_frm_disp_rids():
    return zread.read_hdfs(os.path.join(TARGET_PATH, 'target_frm_disp'))

